This notebook takes all of the preliminary SNeCO data and organizes it, preparing it for when we begin making intensity measurements in section 3.

To run this notebook we need '2.SNeCOMasterCatManualClean.txt' (see prepwork below). Once this prepwork is done, it is not needed to repeat unless we update the sample / redownload the OSC.

We also need "0.PHANGS_v1p6.csv" to pull other galaxy information from the survey.

      *PREPWORK* 
    After looking at the galaxy images from #2. We find we need to remove one SN that isnt within the region covered by the PHANGS-ALMA footprint. We remove this (SN2003gd from NGC0628) and also the 6 candidate SNe, the one duplicate entry, and the LRN, and begin to perform some rudimentary stats in small tables.

    Before we run this code we do the following prep:

    We take the 2.SNeCOMasterCat.txt file and manually remove the following SNe:

    AT2020nqq (LRN) 
    AT2019pck (Candidate) 
    AT2019npi (Candidate) 
    AT2019npd (Candidate) 
    AT2020cuh (Candidate) 
    AT2020hol (Candidate) 
    AT2020juh (Candidate) 
    PTSS-19clju (Duplicate) 
    SN2003gd (Out of Map)

    *also shortened type IIn, IIb to II and type Ia-91bg to Ia, etc.

    Then we save this as 2.SNeCOMasterCatManualClean.txt
    
    *MORE PREPWORK* Changed 'name' column to 'galaxy' & 'eso097-013' to 'circinus' in 0.PHANGS_v1p6.csv file
    

In [1]:
# IMPORT PACKAGES

import numpy as np
from matplotlib import pyplot as plt
import astropy.io.fits as pyfits
from astropy.table import Table, join
from astropy.wcs import WCS
from astropy.io import ascii
import astropy
from astropy.io import fits

In [2]:
# Turn file into dataStructure and clean

dataFile = '../Data/2.SNeCOMasterCatManualClean.txt'
galaxy, SNname, SNtype = np.genfromtxt(dataFile,  dtype=str, unpack=True, delimiter = ',', usecols=(0,1,6))
ra, dec, xVal, yVal, co_21_int, co_21_err, co_21_ew, beamsize  = np.genfromtxt(dataFile,  dtype=float, unpack=True, delimiter = ',', usecols=(2,3,4,5,7,8,9,10))

# remove spaces and " " from SN data
galaxy = [x.strip() for x in galaxy]
SNname = [x[2:-2] for x in SNname]
SNtype = [x.strip() for x in SNtype]

# build table 
data = Table({'galaxy'       : galaxy,
              'SN_name'      : SNname,
              'SN_type'      : SNtype,
              'SN_ra'        : ra,
              'SN_dec'       : dec,
              'SN_xCoord'    : xVal,
              'SN_yCoord'    : yVal,
              'SN_co21int'   : co_21_int,
              'SN_co21err'   : co_21_err,
              'SN_co21ew'    : co_21_ew,
              'map_beamsize' : beamsize,
             },
             names=['galaxy', 'SN_name','SN_type','SN_ra','SN_dec','SN_xCoord', 'SN_yCoord', 'SN_co21int', 'SN_co21err', "SN_co21ew", 'map_beamsize'])


In [9]:
# Pull Galaxy data from PHANGS survey

PHANGS_data = '../Data/0.PHANGS_v1p6.csv'
PHANGSTable = Table.read(PHANGS_data, format='csv')

#PHANGS_clean = PHANGSTable['galaxy', 'orient_ra', 'orient_dec', 'dist', 'orient_posang', 'orient_incl', 'size_r25']

 galaxy  survey_alma_status
-------- ------------------
circinus           released
  ic0010 cannot_be_observed
  ic0342 cannot_be_observed
  ic1954           released
  ic1993      not_in_survey
  ic5273           released
  ic5332           released
 ngc0224 cannot_be_observed
 ngc0247           released
 ngc0253           released
     ...                ...
 ngc5248           released
 ngc5457 cannot_be_observed
 ngc5530           released
 ngc5643           released
 ngc6300           released
 ngc6744           released
 ngc6946 cannot_be_observed
 ngc7456           released
 ngc7496           released
 ngc7743           released
 ngc7793           released
Length = 118 rows


In [14]:
for i in range(len(PHANGSTable)):
    print(PHANGSTable['galaxy', 'survey_alma_status'][i])


 galaxy  survey_alma_status
-------- ------------------
circinus           released
galaxy survey_alma_status
------ ------------------
ic0010 cannot_be_observed
galaxy survey_alma_status
------ ------------------
ic0342 cannot_be_observed
galaxy survey_alma_status
------ ------------------
ic1954           released
galaxy survey_alma_status
------ ------------------
ic1993      not_in_survey
galaxy survey_alma_status
------ ------------------
ic5273           released
galaxy survey_alma_status
------ ------------------
ic5332           released
 galaxy survey_alma_status
------- ------------------
ngc0224 cannot_be_observed
 galaxy survey_alma_status
------- ------------------
ngc0247           released
 galaxy survey_alma_status
------- ------------------
ngc0253           released
 galaxy survey_alma_status
------- ------------------
ngc0278 cannot_be_observed
 galaxy survey_alma_status
------- ------------------
ngc0300           released
 galaxy survey_alma_status
------- --------

 galaxy survey_alma_status
------- ------------------
ngc6946 cannot_be_observed
 galaxy survey_alma_status
------- ------------------
ngc7456           released
 galaxy survey_alma_status
------- ------------------
ngc7496           released
 galaxy survey_alma_status
------- ------------------
ngc7743           released
 galaxy survey_alma_status
------- ------------------
ngc7793           released


In [4]:
SN_plus_gal_data = join(data, PHANGS_clean, keys='galaxy')

# SN_plus_gal_data.colnames

In [6]:
ascii.write(SN_plus_gal_data, '../Data/3.SNe+GalData.csv', format='csv', fast_writer=False, overwrite=True)  